## 

In [102]:
import os
%matplotlib inline
import sys
from collections import Counter
from typing import List

In [103]:
# TextRank
from gensim.summarization.summarizer import summarize

In [104]:
# kiwi: Tokenizer
from kiwipiepy import Kiwi
# keyBert
from keybert import KeyBERT
from transformers import BertModel

In [105]:
from tqdm.notebook import tqdm

In [106]:
module_path = '/'.join(os.getcwd().split("\\")[:-1])
sys.path.append(module_path)
sys.path.append(module_path + '/crawling')

## data 전처리

### Tokenize

In [107]:
kiwi = Kiwi()

In [108]:
# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    # print(result)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

### 불용어 제거

#### stopword: list

In [109]:
stopwords = [
    # channel
    "11번가",
    "11st-tech",
    "29CM",
    "29cm",
    "AWS",
    "amazon",
    "class101",
    "GS리테일",
    "gsretail",
    "NHN",
    "nhncloud",
    "ToastUI",
    "네이버D2",
    "d2",
    "naver",
    "네이버클라우드",
    "naver-cloud-platform",
    "네이버플레이스",
    "naver-place-dev",
    "넷마블",
    "netmarble",
    "다나와",
    "danawalab",
    "당근마켓",
    "daangn",
    "데보션",
    "devocean",
    "sk",
    "데브시스터즈",
    "devsisters",
    "드라마앤컴퍼니",
    "dramancompany",
    "라인",
    "linecorp",
    "레모네이드",
    "lemonade-engineering",
    "롯데on",
    "lotteon",
    "루닛",
    "lunit",
    "마이리얼트립",
    "myrealtrip-product",
    "메가존클라우드",
    "ctc-mzc",
    "무신사",
    "musinsa",
    "버즈빌",
    "buzzvil",
    "브랜디",
    "brandi",
    "사람인",
    "saramin",
    "숨고",
    "soomgo",
    "스마일게이트AI",
    "smilegate",
    "스케터랩",
    "scatterlab",
    "스타일쉐어",
    "styleshare",
    "쏘카",
    "socarcorp",
    "아이디어스",
    "idus",
    "야놀자",
    "yanolja",
    "야놀자클라우드",
    "yanoljacloud-tech",
    "엔라이즈",
    "nrise",
    "여기어때",
    "gccompany",
    "오일나우",
    "왓챠",
    "watcha",
    "요기요",
    "yogiyo",
    "우아한형제들",
    "woowahan",
    "원티드",
    "wantedjobs",
    "지마켓",
    "gmarket",
    "직방",
    "zigbang",
    "카카오",
    "kakao",
    "카카오엔터프라이즈",
    "kakaoenterprise",
    "카카오페이",
    "kakaopay",
    "컬리",
    "kurly",
    "코인원",
    "coinone",
    "쿠팡",
    "coupang-engineering",
    "크몽",
    "kmong",
    "클라우드메이트",
    "cloudmt",
    "테이블링",
    "tabling",
    "토스",
    "toss",
    "포스타입",
    "postype",
    "하이퍼커넥트",
    "hyperconnect",
    "헤이딜러",
    "prnd",
    "화해",
    "hwahae",
]

#### stopword: from DB

In [110]:
# from crawling import config
# #  from src.crawling.config import connect, close

# conn = config.connect()
# curs = conn.cursor()

# channel_sql = """select name from channel group by name"""
# curs.execute(channel_sql)
# channel_info = curs.fetchall()

# channels = []
# for channel in channel_info:
#     channels.extend(channel)
# print(channels)

# config.close()

# stopwords.extend(channels)

## keyword 추출

In [111]:
def keyword_extractor(bert:str, documents:List[str]): # -> (List[Tuple[str, float]] | List[List[Tuple[str, float]]]):
  model = BertModel.from_pretrained(bert)
  model = model.cuda()
  kw_model = KeyBERT(model)
  keywords = kw_model.extract_keywords(documents, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
  return keywords
# model_100langs = BertModel.from_pretrained('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
# kw_model_100langs = KeyBERT(model_100langs)
# keywords_100langs = kw_model_100langs.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)


In [112]:
import torch
print(torch.cuda.is_available())

True


In [113]:
# gpu 사용법
# https://github.com/MaartenGr/KeyBERT/issues/108
# model = SentenceTransformer(
#     "<hf_model_name>",
#     device="cuda:0"
# )
model = BertModel.from_pretrained('skt/kobert-base-v1')
model = model.cuda()
print("DEVICE", model.device)
# hf_model = KeyBERT(model)

# https://www.kaggle.com/code/accountstatus/shopee-competition-using-bert-model


DEVICE cuda:0


In [114]:
# print(os.getcwd())
# base_path = "/content/drive/MyDrive/쳇, 6pt: 비스킷(biskuit)/dataset/content/"
import csv
import datetime

base_path = '../../../data/content/'
csv_path = '../../data/crawling_data.csv'

f = open(csv_path, "r", encoding="utf-8")

csvReader = csv.reader(f)

def extract_keyword(base_path:str):
    documents = []

    for row in csvReader:
        if len(row) == 0:
            continue
        title = row[1]
        created_date = row[3]
        channel = row[2]
        categories = row[4].split()
        print(channel + '_' + title + ' '.join(categories))
        # "Design", "Product", "Culture", "Conference" 태그만 달려있는 글 분석 제외
        if len(categories) != 0 and len(set(categories) - set(["Design", "Product", "Culture", "Conference"])) == 0: continue
        content_path = base_path + created_date + '/' + channel + '_' + title + '.txt'
        file = open(content_path, 'r', encoding='utf-8')
        text = file.read()
        if len(text.split('\n')) > 50:
            summary = summarize(text)
        else:
            summary = text
        # nouns = noun_extractor(text)
        nouns = noun_extractor(summary)
        text = ' '.join(nouns)
        documents.append(text)
    return keyword_extractor('skt/kobert-base-v1', documents)

In [116]:
def extract_keyword2(base_path:str):
  content_dir_list = os.scandir(base_path)
  documents = []
  for content_dir in content_dir_list:
      if not content_dir.is_dir(): continue
      content_list = os.scandir(content_dir)
      for content in content_list:
          if not content.is_file(): continue
          file = open(content.path, "r")
          if len(text.split('\n')) > 50:
            summary = summarize(text)
          else:
            summary = text
          # nouns = noun_extractor(text)
          nouns = noun_extractor(summary)
          text = ' '.join(nouns)
          documents.append(text)
  # print(text)
  keyword_extractor('skt/kobert-base-v1', documents)
  # keyword_extractor('paraphrase-multilingual-MiniLM-L12-v2', documents)   # multilingual!
  # keyword_extractor('paraphrase-multilingual-mpnet-base-v2', documents)   # 시간 더 오래걸리고 결과 더 조음!

In [117]:
faster_keywords = extract_keyword(base_path)
faster_counter = Counter(faster_keywords)
print(faster_counter.most_common(100))

브랜디_Jest를 이용한 vuejs 프로젝트에 Unit Test 도입
네이버클라우드_Ncloud Elasticsearch Service 활용 아파치 로그 분석하기
스케터랩_Transformer - Harder  Better  Faster  Stronger
네이버클라우드_[NBP 기술&경험] 시대의 흐름  gRPC 깊게 파고들기 #2


FileNotFoundError: [Errno 2] No such file or directory: '../../../data/content/2020-01-03/네이버클라우드_[NBP 기술&경험] 시대의 흐름  gRPC 깊게 파고들기 #2.txt'